In [3]:
import os
import math
import pickle
import pandas as pd
from datetime import datetime
from collections import Counter
from konlpy.tag import Twitter; tw = Twitter()

C:\Users\andrew\.conda\envs\text3.5\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
# def make_load_dic(pic_list):
#     news_ = {} # 
#     for i in pic_list[:]:
#         with open(i, 'rb') as f:
#             news_.update(pickle.load(f))
#     return news_

In [5]:
pic_list_2015 = []
pic_list_2014 = []
pic_list_2013 = []
for (path, dir_, files) in os.walk('./중앙일보/금융증권/'):
    if files != []:
        for pic in files:
            ##### 목표년도
            if pic[5:9] == '2015':
                pic_list_2015.append('./중앙일보/금융증권/'+pic[:])
            if pic[5:9] == '2014':
                pic_list_2014.append('./중앙일보/금융증권/'+pic[:])
            if pic[5:9] == '2013':
                pic_list_2013.append('./중앙일보/금융증권/'+pic[:])

In [6]:
##### 날짜 지정
index_date = [i.strftime('%Y.%m.%d') for i in pd.date_range('20150101', '20151231')]

In [7]:
# 2013년도
news_ = {}

##### pic_list_년도 만 바꾸면된다.
for i in pic_list_2015[:]:
    # load
    with open(i, 'rb') as f:
        news_.update(pickle.load(f))

In [8]:
# 일별로
news_each_day = {}
for idx in index_date[:]:
    news_day = ''
    for find_date in news_.keys():
        if news_[find_date]['date'][:-6] == idx:
            news_day += '--------------------------news start-------------------------------'+news_[find_date]['script'].replace('\n','').replace('\u3000','').replace('\r','')
    news_each_day[idx] = {'date':idx, 'script':news_day}

In [9]:
news_each_day['2015.01.01']

{'date': '2015.01.01',
 'script': '--------------------------news start--------------------------------4.76%.지난해 국내 증시 성적표다. 지난해 마지막 거래일인 30일 코스피는 1915.59로 한해를 마쳤다. 2013년 폐장일(2011.34)보다 4.76% 하락했다.국내 증시는 일년간 지루하다고 할 정도로 박스권에서만 맴돌았다. 조금 상승하는 분위기다 싶으면 다시 악재가 돌출해 고꾸라지기 일쑤였다. 상반기 지리멸렬했던 코스피는 하반기 최경환 경제부총리 취임후 경기 부양에 대한 기대감으로 상승세를 탔다. 정부의 기업에 대한 배당·투자 확대 유도, 부동산 규제 완화, 한국은행의 기준금리 인하 등으로 7월엔 2093선까지 오르며 2100선 탈환 초읽기에 들어가는 듯했다. 당시 증권가에서는 박스권에서만 머물던 코스피의 대세 상승이 시작됐다는 기대감까지 피어올랐다.하지만 국회 공전으로 경기부양 관련법 통과가 지연됐고, 일본의 엔화 약세로 국내 기업 경쟁력 약화에 대한 우려의 목소리는 커졌다. 시장에서는 “법 집행의 ‘골든 타임’을 놓쳤다”는 지적이 제기됐다. 여기에 국내 기업 실적 부진, 유럽 경기 둔화, 유가 하락에 따른 러시아·브라질 등의 경제 불안까지 겹치며 코스피는 미끄러지길 거듭했다. 결국 12월 한때 1880선까지 떨어지며 1900선을 방어하는데 급급해야 했다.이와 달리 경기 회복세를 보이는 미국은 기업 실적 개선을 바탕으로 증시가 연일 사상 최고치를 경신했다. 홍콩과 상하이 증시의 교차 거래를 허용하는 ‘후강퉁’ 실시로 중국증시는 50% 가량 상승했고 일본도 시장에 돈을 푸는 ‘아베노믹스’ 정책에 힘입어 고공행진을 거듭했다.이 때문에 한국(코스피 기준)은 지난해 G20 국가 가운데 증시 상승률이 꼴찌에서 두 번째(19위)에 머물렀다. 아르헨티나가 56.6%로 1위에 올랐고 중국(49.7%), 인도(29.4%), 터키(24.0%), 인도네시아(21.1%), 일본(8.8%),

In [10]:
# news_each_day['2014.08.18']

In [11]:
token_ = {}
for i in list(news_each_day.keys())[:]:
#     print(i)
#     print(news_2013[i]['date'])
    token_[i] = {'date' : datetime.strptime(news_each_day[i]['date'], '%Y.%m.%d'), 'nouns' : tw.nouns(news_each_day[i]['script'])}

In [12]:
finish_conunt = 0
tf_idf_days = {}
for idx_day in token_.keys():

    # TF
    count = Counter(token_[idx_day]['nouns'])
    TF = {}
    for cnoun in count.keys():
        TF[cnoun] = count[cnoun]/len(token_[idx_day]['nouns'])

    # DF
    DF = {}
    for cnoun in count.keys():
        DF[cnoun] = 0
    for cnoun in count.keys():
        for i1 in list(token_.keys()):
            df_value = set(token_[i1]['nouns']) & {cnoun}
            if df_value != set():
                DF[cnoun] += 1
                
    # TF-IDF           
    TF_IDF = {}
    for i2 in count.keys():
        # IDF
        IDF = math.log(len(token_)/(1+DF[i2]))
        # TF * IDF
        TF_IDF[i2] = TF[i2] * IDF
#     result = list(TF_IDF.values())

    # sort and reverse
    sorted_by_value = sorted(TF_IDF.items(), key=lambda kv: kv[1])
    sorted_by_value.reverse()
    
    # select top30
    tf_idf_days[idx_day] = sorted_by_value[:30]

    finish_conunt += 1
    print(finish_conunt,'/365')
    if finish_conunt == 10:
        break

1 /365
2 /365
3 /365
4 /365
5 /365
6 /365
7 /365
8 /365
9 /365
10 /365


In [13]:
# tf_idf_days

group = '사전'

with open('./중앙일보/'+group+'/news_2015_test.pickle', 'wb') as f:
    pickle.dump(tf_idf_days, f, pickle.HIGHEST_PROTOCOL)

In [14]:
with open('./중앙일보/사전/news_2015_test.pickle', 'rb') as f:
    data2 = pickle.load(f)

In [15]:
data2

{'2015.03.22': [],
 '2015.04.09': [('제주도', 0.019293502308273102),
  ('동결', 0.014613376786456729),
  ('기준금리', 0.014015723239625156),
  ('가치투자', 0.013124398773539835),
  ('알리페', 0.012402965769604138),
  ('인하', 0.011288329939466208),
  ('임노', 0.010479872874248666),
  ('하방', 0.010479872874248666),
  ('지표', 0.010104379682012068),
  ('방천', 0.009663774032031629),
  ('감정', 0.00965397453361542),
  ('통화정책', 0.009609079289956319),
  ('가계부채', 0.009534461163843135),
  ('망치', 0.008735526627233016),
  ('박경', 0.008733227395207223),
  ('분양', 0.008651473402557484),
  ('언급', 0.007975916585055044),
  ('치가', 0.007646230698545507),
  ('아파트', 0.007308046643476454),
  ('수치', 0.007268243373473562),
  ('자문', 0.007262257272539357),
  ('예상', 0.007075239418267222),
  ('경제지표', 0.006873513660773883),
  ('금통위', 0.006873513660773883),
  ('팀', 0.006767504021629427),
  ('투자', 0.0067244347706188635),
  ('이채원', 0.006631981221950987),
  ('연내', 0.0065280273792996),
  ('명', 0.006327275281426279),
  ('부사', 0.00629558896377055